##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 그래프 및 tf.function 소개

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/guide/intro_to_graphs"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org에서 보기</a>   </td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ko/guide/intro_to_graphs.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/guide/intro_to_graphs.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub에서 소스 보기</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ko/guide/intro_to_graphs.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운로드</a></td>
</table>

## 개요

이 가이드는 TensorFlow의 TensorFlow 및 Keras의 내부를 살펴봄으로써 TensorFlow의 동작 방식을 설명합니다. 대신 Keras를 바로 시작하려면 [Keras 가이드 모음](https://www.tensorflow.org/guide/keras/)을 확인하세요.

이 가이드에서는 TensorFlow를 사용하여 코드를 간단하게 변경하고 그래프를 가져오는 방법, 그래프를 저장하고 표시하는 방법, 그리고 이를 사용하여 모델을 가속화하는 방법을 배웁니다.

참고: TensorFlow 1.x에만 익숙한 사용자를 위해 이 가이드는 매우 다른 그래프 뷰를 보여줍니다.

**This is a big-picture overview that covers how `tf.function` allows you to switch from eager execution to graph execution.** For a more complete specification of `tf.function`, go to the <a href="./function.ipynb" data-md-type="link">Better performance with `tf.function`</a> guide.


### 그래프란 무엇인가요?

In the previous three guides, you ran TensorFlow **eagerly**. This means TensorFlow operations are executed by Python, operation by operation, and returning results back to Python.

즉시 실행에는 몇 가지 고유한 장점이 있지만 그래프 실행은 Python 외부에서 이식성을 가능하게 하며 성능이 더 우수한 경향이 있습니다. **그래프 실행**은 텐서 계산이 `tf.Graph` 또는 간단히 "그래프"라고도 하는 *TensorFlow 그래프*로 실행됨을 의미합니다.

**그래프는 계산의 단위를 나타내는 `tf.Operation` 객체와 연산 간에 흐르는 데이터의 단위를 나타내는 `tf.Tensor` 객체의 세트를 포함합니다.** 데이터 구조는 `tf.Graph` 컨텍스트에서 정의됩니다. 그래프는 데이터 구조이므로 원래 Python 코드 없이 모두 저장, 실행 및 복원할 수 있습니다.

이것은 TensorBoard에서 시각화했을 때 2계층 신경망을 나타내는 TensorFlow 그래프의 모습입니다.

<img alt="A simple TensorFlow graph" src="https://github.com/tensorflow/docs/blob/master/site/en/guide/images/intro_to_graphs/two-layer-network.png?raw=1" class="">  

### 그래프의 이점

그래프를 사용하면 유연성이 크게 향상됩니다. 모바일 애플리케이션, 임베디드 기기 및 백엔드 서버와 같은 Python 인터프리터가 없는 환경에서 TensorFlow 그래프를 사용할 수 있습니다. TensorFlow는 그래프를 Python에서 내보낼 때 저장된 모델의 형식으로 그래프를 사용합니다.

그래프는 쉽게 최적화되어 컴파일러가 다음과 같은 변환을 수행할 수 있습니다.

- 계산에서 상수 노드를 접어 텐서의 값을 정적으로 추론합니다*("일정한 접기")*.
- 독립적인 계산의 하위 부분을 분리하여 스레드 또는 기기 간에 분할합니다.
- 공통 하위 표현식을 제거하여 산술 연산을 단순화합니다.


위와 같은 변환 및 기타 속도 향상을 수행하기 위한 전체 최적화 시스템으로 [Grappler](./graph_optimization.ipynb)가 있습니다.

요약하면, 그래프는 TensorFlow가 **빠르게**, **병렬로**, 그리고 효율적으로 **여러 기기에서** 실행할 때 아주 유용합니다.

그러나 편의를 위해 Python에서 머신러닝 모델(또는 기타 계산)을 정의한 다음 필요할 때 자동으로 그래프를 구성하려고 합니다.

## 설정하기

Import some necessary libraries:

In [ ]:
import tensorflow as tf
import timeit
from datetime import datetime

## 그래프 이용하기

`tf.function`을 직접 호출 또는 데코레이터로 사용하여 TensorFlow에서 그래프를 만들고 실행합니다. `tf.function`은 일반 함수를 입력으로 받아 `Function`을 반환합니다. <strong data-md-type="raw_html">`Function`은 Python 함수로부터 TensorFlow 그래프를 빌드하는 Python callable입니다. Python의 경우와 동일한 방식으로 `Function`를 사용합니다.</strong>


In [ ]:
# Define a Python function.
def a_regular_function(x, y, b):
  x = tf.matmul(x, y)
  x = x + b
  return x

# `a_function_that_uses_a_graph` is a TensorFlow `Function`.
a_function_that_uses_a_graph = tf.function(a_regular_function)

# Make some tensors.
x1 = tf.constant([[1.0, 2.0]])
y1 = tf.constant([[2.0], [3.0]])
b1 = tf.constant(4.0)

orig_value = a_regular_function(x1, y1, b1).numpy()
# Call a `Function` like a Python function.
tf_function_value = a_function_that_uses_a_graph(x1, y1, b1).numpy()
assert(orig_value == tf_function_value)

On the outside, a `Function` looks like a regular function you write using TensorFlow operations. [Underneath](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/eager/def_function.py), however, it is *very different*. A `Function` **encapsulates several `tf.Graph`s behind one API** (learn more in the *Polymorphism* section). That is how a `Function` is able to give you the benefits of graph execution, like speed and deployability (refer to *The benefits of graphs* above).

`tf.function`은 함수 및 *이 함수가 호출하는 다른 모든 함수*에 적용됩니다.

In [ ]:
def inner_function(x, y, b):
  x = tf.matmul(x, y)
  x = x + b
  return x

# Use the decorator to make `outer_function` a `Function`.
@tf.function
def outer_function(x):
  y = tf.constant([[2.0], [3.0]])
  b = tf.constant(4.0)

  return inner_function(x, y, b)

# Note that the callable will create a graph that
# includes `inner_function` as well as `outer_function`.
outer_function(tf.constant([[1.0, 2.0]])).numpy()

TensorFlow 1.x를 사용한 경우 `Placeholder` 또는 `tf.Sesssion`을 정의할 필요가 없었음을 알 수 있습니다.

### Python 함수를 그래프로 변환하기

TensorFlow를 사용하여 작성하는 모든 함수에는 `if-then` 절, 루프, `break`, `return`, `continue` 등과 같은 내장된 TF 연산과 Python 논리가 혼합되어 있습니다. TensorFlow 연산은 `tf.Graph`에 의해 쉽게 캡처되지만 Python 관련 논리는 그래프의 일부가 되기 위해 추가 단계를 거쳐야 합니다. `tf.function`은 AutoGraph(`tf.autograph`)라는 라이브러리를 사용하여 Python 코드를 그래프 생성 코드로 변환합니다.


In [ ]:
def simple_relu(x):
  if tf.greater(x, 0):
    return x
  else:
    return 0

# `tf_simple_relu` is a TensorFlow `Function` that wraps `simple_relu`.
tf_simple_relu = tf.function(simple_relu)

print("First branch, with graph:", tf_simple_relu(tf.constant(1)).numpy())
print("Second branch, with graph:", tf_simple_relu(tf.constant(-1)).numpy())

그래프를 직접 볼 필요는 없겠지만 결과를 검사하여 정확한 결과를 확인할 수 있습니다. 읽기가 쉽지 않으므로 너무 주의 깊게 볼 필요는 없습니다!

In [ ]:
# This is the graph-generating output of AutoGraph.
print(tf.autograph.to_code(simple_relu))

In [ ]:
# This is the graph itself.
print(tf_simple_relu.get_concrete_function(tf.constant(1)).graph.as_graph_def())

Most of the time, `tf.function` will work without  special considerations. However, there are some caveats, and the <a href="./function.ipynb" data-md-type="link">`tf.function` guide</a> can help here, as well as the [complete AutoGraph reference](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/autograph/g3doc/reference/index.md).

### 다형성: 하나의 `Function`, 다수의 그래프

`tf.Graph`는 특정 유형의 입력에 특화되어 있습니다(예: 특정 [`dtype`](https://www.tensorflow.org/api_docs/python/tf/dtypes/DType)을 가진 텐서 또는 동일한 [`id()`](https://docs.python.org/3/library/functions.html#id%5D)를 가진 객체).

Each time you invoke a `Function` with a set of arguments that can't be handled by any of its existing graphs (such as arguments with new `dtypes` or incompatible shapes), `Function` creates a new `tf.Graph` specialized to those new arguments. The type specification of a `tf.Graph`'s inputs is known as its **input signature** or just a **signature**. For more information regarding when a new `tf.Graph` is generated and how that can be controlled, go to the *Rules of tracing* section of the [Better performance with `tf.function`](./function.ipynb) guide.

`Function`은 해당 서명에 대응하는 `tf.Graph`를 `ConcreteFunction`에 저장합니다. <strong data-md-type="raw_html">`ConcreteFunction`은 `tf.Graph`를 감싸는 래퍼입니다.</strong>


In [ ]:
@tf.function
def my_relu(x):
  return tf.maximum(0., x)

# `my_relu` creates new graphs as it observes more signatures.
print(my_relu(tf.constant(5.5)))
print(my_relu([1, -1]))
print(my_relu(tf.constant([3., -3.])))

`Function`이 이 서명으로 이미 호출된 경우, `Function`은 새 `tf.Graph`를 생성하지 않습니다.

In [ ]:
# These two calls do *not* create new graphs.
print(my_relu(tf.constant(-2.5))) # Signature matches `tf.constant(5.5)`.
print(my_relu(tf.constant([-1., 1.]))) # Signature matches `tf.constant([3., -3.])`.

Because it's backed by multiple graphs, a `Function` is **polymorphic**. That enables it to support more input types than a single `tf.Graph` could represent, and to optimize each `tf.Graph` for better performance.

In [ ]:
# There are three `ConcreteFunction`s (one for each graph) in `my_relu`.
# The `ConcreteFunction` also knows the return type and shape!
print(my_relu.pretty_printed_concrete_signatures())

## `tf.function` 사용하기

지금까지 `tf.function`을 데코레이터 또는 래퍼로 사용하여 Python 함수를 간단히 그래프로 변환하는 방법을 살펴보았습니다. 그러나 실제로 `tf.function`이 올바르게 작동하도록 만드는 일은 까다로울 수 있습니다! 다음 섹션에서는 `tf.function`을 사용하여 코드가 예상대로 작동하도록 만드는 방법을 알아봅니다.

### 그래프 실행 vs 즉시 실행

`Function`의 코드는 즉시 실행 또는 그래프 실행이 가능합니다. 기본적으로 `Function`은 코드를 그래프로 실행합니다.


In [ ]:
@tf.function
def get_MSE(y_true, y_pred):
  sq_diff = tf.pow(y_true - y_pred, 2)
  return tf.reduce_mean(sq_diff)

In [ ]:
y_true = tf.random.uniform([5], maxval=10, dtype=tf.int32)
y_pred = tf.random.uniform([5], maxval=10, dtype=tf.int32)
print(y_true)
print(y_pred)

In [ ]:
get_MSE(y_true, y_pred)

To verify that your `Function`'s graph is doing the same computation as its equivalent Python function, you can make it execute eagerly with `tf.config.run_functions_eagerly(True)`. This is a switch that <strong data-md-type="double_emphasis">turns off `Function`'s ability to create and run graphs</strong>, instead of executing the code normally.

In [ ]:
tf.config.run_functions_eagerly(True)

In [ ]:
get_MSE(y_true, y_pred)

In [ ]:
# Don't forget to set it back when you are done.
tf.config.run_functions_eagerly(False)

그러나 `Function`은 그래프 및 즉시 실행에서 서로 다르게 동작할 수 있습니다. Python [`print`](https://docs.python.org/3/library/functions.html#print) 함수는 이 두 모드가 어떻게 다른지 보여주는 한 가지 예제입니다. `print` 문을 삽입하고 이를 반복적으로 호출할 때 어떤 일이 발생하는지 살펴보겠습니다.

In [ ]:
@tf.function
def get_MSE(y_true, y_pred):
  print("Calculating MSE!")
  sq_diff = tf.pow(y_true - y_pred, 2)
  return tf.reduce_mean(sq_diff)

인쇄된 내용을 잘 살펴봅니다.

In [ ]:
error = get_MSE(y_true, y_pred)
error = get_MSE(y_true, y_pred)
error = get_MSE(y_true, y_pred)

출력 결과가 놀랍지 않나요? **`get_MSE`는 *세 번* 호출되었지만 한 번만 인쇄되었습니다.**

To explain, the `print` statement is executed when `Function` runs the original code in order to create the graph in a process known as "tracing" (refer to the *Tracing* section of the [`tf.function` guide](./function.ipynb). <strong data-md-type="double_emphasis">Tracing captures the TensorFlow operations into a graph, and `print` is not captured in the graph.</strong>  That graph is then executed for all three calls **without ever running the Python code again**.

실제로 그런지 검사하기 위해 그래프 실행을 해제하고 비교해 보겠습니다.

In [ ]:
# Now, globally set everything to run eagerly to force eager execution.
tf.config.run_functions_eagerly(True)

In [ ]:
# Observe what is printed below.
error = get_MSE(y_true, y_pred)
error = get_MSE(y_true, y_pred)
error = get_MSE(y_true, y_pred)

In [ ]:
tf.config.run_functions_eagerly(False)

`print`는 *Python의 부작용*이며 함수를 `Function`으로 변환할 때 알고 있어야 하는 <a>다른 차이점들</a>이 있습니다. [tf.function으로 성능 향상하기](./function.ipynb) 가이드의 <em>한계</em> 섹션에서 자세히 알아보세요.

참고: 즉시 및 그래프 실행 모두에서 값을 인쇄하려면 `tf.print`를 대신 사용하세요.

### 비평가(Non-strict) 실행

<a id="non-strict"></a>

그래프 실행은 다음을 포함하여 관찰 가능한 효과를 생성하는 데 필요한 작업만 실행합니다.

- 함수의 반환 값
- 다음과 같은 문서화된 잘 알려진 부작용:
    - `tf.print`와 같은 입력/출력 작업
    - `tf.debugging`의 어설션 기능과 같은 디버깅 작업
    - `tf.Variable`의 변형

이 동작은 일반적으로 "비평가 실행"으로 알려져 있으며 필요하거나 필요하지 않은 모든 프로그램 작업을 단계별로 실행하는 즉시 실행과 구분됩니다.

특히 런타임 오류 검사는 관찰 가능한 효과로 간주되지 않습니다. 작업이 불필요하다는 이유로 건너뛰면 런타임 오류가 발생할 수 없습니다.

다음 예제에서는 그래프 실행 중에 "불필요한" 작업인 `tf.gather`을 건너뛰므로 즉시 실행에서와 마찬가지로 런타임 오류 `InvalidArgumentError`가 발생하지 않습니다. 그래프를 실행하는 동안 발생하는 오류에 의존하지 마세요.

In [ ]:
def unused_return_eager(x):
  # Get index 1 will fail when `len(x) == 1`
  tf.gather(x, [1]) # unused 
  return x

try:
  print(unused_return_eager(tf.constant([0.0])))
except tf.errors.InvalidArgumentError as e:
  # All operations are run during eager execution so an error is raised.
  print(f'{type(e).__name__}: {e}')

In [ ]:
@tf.function
def unused_return_graph(x):
  tf.gather(x, [1]) # unused
  return x

# Only needed operations are run during graph execution. The error is not raised.
print(unused_return_graph(tf.constant([0.0])))

### `tf.function` best practices

`Function`의 동작에 익숙해지려면 시간이 걸릴 수 있습니다. 빠르게 시작하려는 처음 사용자는 `@tf.function`으로 시험용 함수를 사용해 보면서 즉시 실행에서 그래프 실행으로 이동하는 과정을 체험할 수 있습니다.

*Designing for `tf.function`* may be your best bet for writing graph-compatible TensorFlow programs. Here are some tips:

- `tf.config.run_functions_eagerly`로 즉시 실행과 그래프 실행 사이를 조기에 자주 전환하여 두 모드가 서로 달라지는지, 언제 달라지는지 정확하게 파악합니다.
- Create `tf.Variable`s outside the Python function and modify them on the inside. The same goes for objects that use `tf.Variable`, like `tf.keras.layers`, `tf.keras.Model`s and `tf.keras.optimizers`.
- Avoid writing functions that depend on outer Python variables, excluding `tf.Variable`s and Keras objects. Learn more in *Depending on Python global and free variables* of the [`tf.function` guide](./function.ipynb).
- Prefer to write functions which take tensors and other TensorFlow types as input. You can pass in other object types but be careful! Learn more in *Depending on Python objects* of the [`tf.function` guide](./function.ipynb).
- 성능 이점을 극대화하기 위해 `tf.function` 하에서 계산이 가능한 한 많이 포함되도록 합니다. 예를 들어 전체 훈룬 스텝 또는 전체 훈룬 루프를 데코레이션합니다.


## 속도 향상 확인하기

`tf.function`은 일반적으로 코드의 성능을 향상시키지만 속도 향상의 정도는 실행하는 계산의 종류에 따라 다릅니다. 작은 계산의 경우 그래프를 호출하는 오버헤드에 의해 지배될 수 있습니다. 다음과 같이 성능 차이를 측정할 수 있습니다.

In [ ]:
x = tf.random.uniform(shape=[10, 10], minval=-1, maxval=2, dtype=tf.dtypes.int32)

def power(x, y):
  result = tf.eye(10, dtype=tf.dtypes.int32)
  for _ in range(y):
    result = tf.matmul(x, result)
  return result

In [ ]:
print("Eager execution:", timeit.timeit(lambda: power(x, 100), number=1000))

In [ ]:
power_as_graph = tf.function(power)
print("Graph execution:", timeit.timeit(lambda: power_as_graph(x, 100), number=1000))

`tf.function` is commonly used to speed up training loops, and you can learn more about it in the <em data-md-type="emphasis">Speeding-up your training step with `tf.function`</em> section of the [Writing a training loop from scratch](https://www.tensorflow.org/guide/keras/writing_a_training_loop_from_scratch) with Keras guide.

Note: You can also try `tf.function(jit_compile=True)` for a more significant performance boost, especially if your code is heavy on TensorFlow control flow and uses many small tensors. Learn more in the <em data-md-type="emphasis">Explicit compilation with `tf.function(jit_compile=True)`</em> section of the [XLA overview](https://www.tensorflow.org/xla).

### 성능과 상충 관계

그래프는 코드의 속도를 높일 수 있지만 이를 생성하는 프로세스에는 약간의 오버헤드가 있습니다. 일부 함수의 경우 그래프를 생성하는 데 그래프를 실행하는 것보다 더 많은 시간이 걸립니다. **이러한 투자를 할 경우 후속 실행에서 성능이 향상되는 보상이 빠르게 뒤따르지만 대규모 모델 훈련의 처음 몇 단계에서는 트레이싱으로 인해 느려질 수 있다는 점을 알고 있어야 합니다**.

No matter how large your model, you want to avoid tracing frequently. The [`tf.function` guide](./function.ipynb) discusses how to set input specifications and use tensor arguments to avoid retracing in the *Controlling retracing* section. If you find you are getting unusually poor performance, it's a good idea to check if you are retracing accidentally.

## `Function`은 언제 트레이싱합니까?

`Function`이 트레이싱을 수행하는 경우를 알아보려면 코드에 `print` 문을 추가합니다. 대략적인 규칙으로, `Function`은 트레이싱할 때마다 `print` 문을 실행합니다.

In [ ]:
@tf.function
def a_function_with_python_side_effect(x):
  print("Tracing!") # An eager-only side effect.
  return x * x + tf.constant(2)

# This is traced the first time.
print(a_function_with_python_side_effect(tf.constant(2)))
# The second time through, you won't see the side effect.
print(a_function_with_python_side_effect(tf.constant(3)))

In [ ]:
# This retraces each time the Python argument changes,
# as a Python argument could be an epoch count or other
# hyperparameter.
print(a_function_with_python_side_effect(2))
print(a_function_with_python_side_effect(3))

새 Python 인수는 항상 새 그래프 생성을 트리거하므로 추가 트레이싱이 발생합니다.


## 다음 단계

API 참조 페이지와 <a><code>tf.function</code>으로 성능 향상하기</a> 가이드에서 `tf.function`에 대한 자세한 내용을 확인할 수 있습니다.